### EDA

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision import models

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score

In [2]:
!pip install GPUtil

In [3]:
from GPUtil import showUtilization as gpu_usage

gpu_usage()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [4]:
path_1 = '../input/skin-cancer-mnist-ham10000/HAM10000_images_part_1'
path_2 = '../input/skin-cancer-mnist-ham10000/HAM10000_images_part_2'

files_path_1 = []
for dirname, _, filenames in os.walk(path_1):
    for filename in filenames:
        files_path_1.append(filename[:-4])

In [5]:
data = pd.read_csv('../input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')

In [6]:
path_list = [(path_1 + '/' + i + '.jpg') if i in files_path_1 else (path_2 + '/' + i + '.jpg') for i in list(data['image_id'])]

In [7]:
data['path'] = path_list

In [8]:
label_types = list(set(data['dx']))

In [9]:
sorted([(l, len(data[data['dx'] == l])/len(data)) for l in label_types], key=lambda x: x[1], reverse=True)

[('nv', 0.6694957563654518),
 ('mel', 0.11113330004992511),
 ('bkl', 0.10973539690464304),
 ('bcc', 0.05132301547678482),
 ('akiec', 0.032651023464802795),
 ('vasc', 0.014178731902146779),
 ('df', 0.011482775836245632)]

In [10]:
labels_dict = {
    'nv': 0,
    'mel': 1,
    'bkl': 2,
    'bcc': 3,
    'akiec': 4,
    'vasc': 5,
    'df': 6
}

In [11]:
data['label'] = [labels_dict[i] for i in data['dx']]

In [12]:
data = data.drop(columns=['lesion_id', 'dx', 'dx_type', 'age', 'sex', 'localization'])

In [13]:
train, test = train_test_split(data, stratify=list(data['label']), test_size = 0.2)

### Dataset

In [14]:
class CustomDataset(Dataset):
    
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        
    def __getitem__(self, index):
        image_path = self.images[index]
        label = self.labels[index]
        image = Image.open(image_path)
        image = image.resize((224, 224))
        image = transforms.ToTensor()(image)
        image = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(image)
        return image, label
        
    def __len__ (self):
        return len(self.images)

In [15]:
train_dataset = CustomDataset(list(train['path']), list(train['label']))
test_dataset = CustomDataset(list(test['path']), list(test['label']))

In [16]:
BATCH_SIZE = 100

In [17]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

### Helpers

In [18]:
def train_nn(model, data_loader, device):
        loss_fn = torch.nn.CrossEntropyLoss()
        model.train()
        for data in tqdm(data_loader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

In [19]:
def eval_nn(model, data_loader, device):
    predicted = []
    labels = []
    model.eval()
    with torch.no_grad():
        for data in tqdm(data_loader):
            x, y = data
            x = x.to(device)

            outputs = model(x)
            _, predict = torch.max(outputs.data, 1)
            predict = predict.cpu().detach().numpy().tolist()
            predicted += predict
            labels += y
        print(f1_score(labels, predicted, average=None))
    return labels, predicted

### Finetuning

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
device

device(type='cuda')

In [22]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [23]:
for epoch in range(3):
    train_nn(model, train_loader, device)
    eval_nn(model, test_loader, device)

100%|██████████| 21/21 [00:43<00:00,  2.09s/it]


[0.86732819 0.18796992 0.46944444 0.52702703 0.05405405 0.77777778
 0.        ]


100%|██████████| 21/21 [00:31<00:00,  1.49s/it]


[0.87055667 0.07758621 0.55364807 0.43478261 0.3655914  0.65714286
 0.        ]


100%|██████████| 21/21 [00:30<00:00,  1.48s/it]

[0.88513514 0.39664804 0.43887147 0.69642857 0.03030303 0.81632653
 0.33333333]


In [24]:
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 |  3% | 26% |


In [25]:
torch.cuda.empty_cache()

In [26]:
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 | 48% | 10% |


### Dataset balancing

In [27]:
from torch.utils.data.sampler import WeightedRandomSampler
y_train = np.array(list(train['label']))
counts = np.bincount(y_train.astype(int))
labels_weights = 1. / counts
labels_weights = 1. / counts
weights = labels_weights[y_train.astype(int)]
sampler = WeightedRandomSampler(weights, len(weights), replacement=True)

In [28]:
train_loader = DataLoader(dataset=train_dataset, sampler=sampler, batch_size=BATCH_SIZE)

In [29]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [30]:
for epoch in range(3):
    train_nn(model, train_loader, device)
    eval_nn(model, test_loader, device)

100%|██████████| 21/21 [00:31<00:00,  1.51s/it]


[0.71132075 0.40343348 0.39058824 0.64285714 0.32571429 0.65882353
 0.23300971]


100%|██████████| 21/21 [00:32<00:00,  1.53s/it]


[0.82150185 0.28037383 0.42829077 0.42105263 0.29577465 0.72727273
 0.20689655]


100%|██████████| 21/21 [00:31<00:00,  1.51s/it]

[0.87651515 0.47136564 0.58503401 0.63043478 0.32941176 0.81818182
 0.54545455]


In [31]:
torch.cuda.empty_cache()

### Augmentation

In [32]:
class CustomDataset2(Dataset):
    
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        
    def __getitem__(self, index):
        image_path = self.images[index]
        label = self.labels[index]
        image = Image.open(image_path)
        image = image.resize((300, 300))
        image = transforms.RandomCrop(224)(image)
        image = transforms.RandomHorizontalFlip()(image)
        image = transforms.RandomPerspective()(image)
        image = transforms.RandomRotation(180)(image)
        image = transforms.RandomVerticalFlip()(image)
        image = transforms.ToTensor()(image)
        image = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(image)
        return image, label
        
    def __len__ (self):
        return len(self.images)

In [33]:
train_dataset = CustomDataset2(list(train['path']), list(train['label']))
test_dataset = CustomDataset2(list(test['path']), list(test['label']))

In [34]:
train_loader = DataLoader(dataset=train_dataset, sampler=sampler, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [35]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [36]:
for epoch in range(3):
    train_nn(model, train_loader, device)
    eval_nn(model, test_loader, device)

100%|██████████| 21/21 [00:35<00:00,  1.71s/it]


[0.6610338  0.38135593 0.5087108  0.50793651 0.32967033 0.52631579
 0.25225225]


100%|██████████| 21/21 [00:36<00:00,  1.73s/it]


[0.65602322 0.47040971 0.35761589 0.31081081 0.23611111 0.12903226
 0.0610687 ]


100%|██████████| 21/21 [00:36<00:00,  1.73s/it]

[0.78736123 0.38736842 0.50903614 0.647343   0.33684211 0.83018868
 0.21176471]


In [37]:
torch.cuda.empty_cache()

### Classificstion: selfmade network

In [38]:
class SimpleConvNet(torch.nn.Module):

    
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p=0.1)
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 7)
        self.batchnorm1 = nn.BatchNorm1d(120)
        self.batchnorm2 = nn.BatchNorm1d(84)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.reshape(-1, 32 * 5 * 5)
        x = self.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [39]:
class CustomDataset3(Dataset):
    
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        
    def __getitem__(self, index):
        image_path = self.images[index]
        label = self.labels[index]
        image = Image.open(image_path)
        image = image.resize((80, 80))
        image = transforms.RandomCrop(64)(image)
        image = transforms.RandomHorizontalFlip()(image)
        image = transforms.RandomRotation(180)(image)
        image = transforms.RandomVerticalFlip()(image)
        image = transforms.ToTensor()(image)
        image = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(image)
        return image, label
        
    def __len__ (self):
        return len(self.images)

In [40]:
train_dataset = CustomDataset3(list(train['path']), list(train['label']))
test_dataset = CustomDataset3(list(test['path']), list(test['label']))

In [41]:
BATCH_SIZE = 100

train_loader = DataLoader(dataset=train_dataset, sampler=sampler, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [42]:
# Xavier init

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

In [43]:
model = SimpleConvNet()

model.apply(init_weights)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [44]:
torch.cuda.empty_cache()
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 | 39% |  5% |


In [45]:
for epoch in range(30):
    train_nn(model, train_loader, device)
    eval_nn(model, test_loader, device)

100%|██████████| 21/21 [00:27<00:00,  1.30s/it]


[0.62345374 0.28205128 0.06949807 0.03571429 0.14893617 0.1938326
 0.05479452]


100%|██████████| 21/21 [00:27<00:00,  1.29s/it]


[0.68245529 0.34910277 0.39895013 0.33333333 0.17073171 0.22564103
 0.15686275]


100%|██████████| 21/21 [00:27<00:00,  1.31s/it]


...

[0.82392304 0.42910448 0.39361702 0.4610951  0.375      0.46601942
 0.39344262]


100%|██████████| 21/21 [00:27<00:00,  1.32s/it]


[0.76777678 0.42364532 0.43554007 0.4784689  0.43137255 0.47787611
 0.4       ]


100%|██████████| 21/21 [00:28<00:00,  1.35s/it]

[0.82945092 0.42519685 0.50093458 0.48797251 0.3539823  0.66666667
 0.3       ]
